In [1]:
# 匯入需要的模組 [B]
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numpy as np # Use numpy to convert to arrays
import pandas as pd # 引用套件並縮寫為 pd
import pmdarima as pm
import pymysql
from datetime import datetime, timedelta
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
# 匯入需要的模組 [E]

In [2]:
# 計算相對誤差 [B]
def rel_error(actual, pred): # x is a row of df
    return abs(float(actual)-float(pred))/float(actual)
# 計算相對誤差 [E]

In [3]:
# 計算 平均絕對百分比誤差 [B]
def mape(actual, pred): 
    actual, pred = np.array(actual).astype('float32'), np.array(pred).astype('float32')
    return np.mean(np.abs((actual - pred) / actual)) * 100
# 計算 平均絕對百分比誤差 [E]

In [4]:
# 計算相對誤差 [B]
def positive_negative_error(actual, pred, dataframe):
    positive_error_list = [0 for number in range(0, 24)]# 低估
    negative_error_list = [0 for number in range(0, 24)] # 高估
    positive_relative_error_list = [0 for number in range(0, 24)]# 低估
    negative_relative_error_list = [0 for number in range(0, 24)] # 高估
    relative_error_list = [0 for number in range(0, 24)]

    positive_error_count_list = [0 for number in range(0, 24)]
    negative_error_count_list = [0 for number in range(0, 24)]
    error_count_list = [0 for number in range(0, 24)]
    
    for i,data_info in dataframe.iterrows():
        error = actual[i] - pred[i]
        error_rate = (actual[i] - pred[i])/actual[i]
        if error > 0:
            positive_relative_error_list[data_info['data_hour'] - 1] += error_rate
            positive_error_list[data_info['data_hour'] - 1] += error
            positive_error_count_list[data_info['data_hour'] - 1] += 1
        elif error < 0:
            negative_relative_error_list[data_info['data_hour'] - 1] += error_rate
            negative_error_list[data_info['data_hour'] - 1] += error
            negative_error_count_list[data_info['data_hour'] - 1] += 1
        relative_error_list[data_info['data_hour'] - 1] += abs(error_rate)
        error_count_list[data_info['data_hour'] - 1] += 1
    
#     positive_error_all_list = [float(value)/float(positive_error_count_list[i]) for i, value in enumerate(positive_error_list)]
#     negative_error_all_list = [float(value)/float(negative_error_count_list[i]) for i, value in enumerate(negative_error_list)]
#     positive_error_all_count_list = [value/(positive_error_count_list[i] + negative_error_count_list[i]) for i, value in enumerate(positive_error_count_list)]
#     negative_error_all_count_list = [value/(positive_error_count_list[i] + negative_error_count_list[i]) for i, value in enumerate(negative_error_count_list)]
#     positive_error_rate = [value/len(actual) for i, value in enumerate(positive_error_count_list)]
#     negative_error_rate = [value/len(actual) for i, value in enumerate(negative_error_count_list)]
#     positive_relative_error_rate = [value/(positive_error_count_list[i]) for i, value in enumerate(positive_relative_error_list)]
#     negative_relative_error_rate = [value/(negative_error_count_list[i]) for i, value in enumerate(negative_relative_error_list)]
    relative_error_rate = [value/error_count_list[i] for i, value in enumerate(relative_error_list)]

    return relative_error_rate
# 計算相對誤差 [E]

In [5]:
# 計算平均需量 [B]
def demand_avg(actual, dataframe):
    positive_error_list = [0 for number in range(0, 24)]# 低估
    positive_error_count_list = [0 for number in range(0, 24)]
    negative_error_list = [0 for number in range(0, 24)] # 高估
    negative_error_count_list = [0 for number in range(0, 24)]
    for i,data_info in dataframe.iterrows():
        positive_error_list[data_info['data_hour'] - 1] += actual[i]
        positive_error_count_list[data_info['data_hour'] - 1] += 1
    positive_error_all_list = [float(value)/float(positive_error_count_list[i]) for i, value in enumerate(positive_error_list)]
    return positive_error_all_list
# 計算平均需量 [E]

In [6]:
# 計算每個同星期的相對誤差 [B]
def positive_negative_error_weekday(actual, pred, dataframe):
    weekday_holiday_list = {
        1: 0,
        2: 0,
        3: 0,
        4: 0,
        5: 0,
        6: 1,
        7: 1
    }
    relative_error_list = [0, 0]
    positive_relative_error_list = [0,0]# 低估
    negative_relative_error_list = [0,0] # 高估
    relative_error_count_list = [0,0]
    positive_error_count_list = [0,0]
    negative_error_count_list = [0,0]
    
    for i,data_info in dataframe.iterrows():
        error = actual[i] - pred[i]
        error_rate = (actual[i] - pred[i])/actual[i]
        abs_error_rate = abs((actual[i] - pred[i])/actual[i])
        if error > 0:
            positive_relative_error_list[weekday_holiday_list[data_info['data_weekday']]] += error_rate
            positive_error_count_list[weekday_holiday_list[data_info['data_weekday']]] += 1
        elif error < 0:
            negative_relative_error_list[weekday_holiday_list[data_info['data_weekday']]] += error_rate
            negative_error_count_list[weekday_holiday_list[data_info['data_weekday']]] += 1
        relative_error_list[weekday_holiday_list[data_info['data_weekday']]] += abs_error_rate
        relative_error_count_list[weekday_holiday_list[data_info['data_weekday']]] += 1
#     positive_relative_error_rate = [value/(positive_error_count_list[i]) for i, value in enumerate(positive_relative_error_list)]
#     negative_relative_error_rate = [value/(negative_error_count_list[i]) for i, value in enumerate(negative_relative_error_list)]
    
    relative_error_rate = [value/(relative_error_count_list[i]) for i, value in enumerate(relative_error_list)]
    return relative_error_rate
# 計算每個同星期的相對誤差 [E]

In [7]:
# 參數設定 [B]
predict_begin_date = '2021-01-01'
predict_end_date = '2022-01-01'
# predict_begin_date = '2020-07-06'
# predict_end_date = '2020-07-12'
# predict_begin_date = '2020-11-30'
# predict_end_date = '2020-12-06'
# 參數設定 [E]

vanilla_demand_predict_weekdayholiday
==========


In [9]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_weekholiday_24hr` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [10]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 29.468179357022
mean_squared_error: 1847.9155660575298
rmse: 42.9873884535631
r2 score: 0.7236789314274599
MRE: 13.257297179289107
MAPE: 13.25729638338089


vanilla_demand_predict_no_weather
==========


In [39]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_96_noweather` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [40]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 40.26413461538461
mean_squared_error: 2598.2391581521733
rmse: 50.97292573663173
r2 score: 0.6284801563367463
MRE: 17.472781640883117
MAPE: 17.472781240940094


vanilla_demand_predict_1hr
==========


In [88]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_4step` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [89]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 40.35209216279853
mean_squared_error: 2881.094555835856
rmse: 53.675828413130766
r2 score: 0.5672145167657056
MRE: 18.614385127609427
MAPE: 18.614384531974792


predict_2
-------------

In [90]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 40.2218472409152
mean_squared_error: 2866.8654286339165
rmse: 53.5431174721263
r2 score: 0.5694634879655995
MRE: 18.561879818419687
MAPE: 18.56187880039215


predict_3
-------------

In [91]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 40.15150790979468
mean_squared_error: 2849.8973968697405
rmse: 53.38443028514719
r2 score: 0.5721084410462596
MRE: 18.553022908811577
MAPE: 18.553023040294647


predict_4
-------------

In [92]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 40.215632996633
mean_squared_error: 2848.0940431986533
rmse: 53.367537353700826
r2 score: 0.5724629723775773
MRE: 18.59823949338063
MAPE: 18.59823763370514


vanilla_demand_predict_12hr
==========


In [96]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_48step` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [97]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 32.275786819098855
mean_squared_error: 1941.0693012441156
rmse: 44.05756803596989
r2 score: 0.7083854237850401
MRE: 14.483789677949105
MAPE: 14.4837886095047


predict_2
-------------

In [98]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 32.00454759502186
mean_squared_error: 1900.1014577194753
rmse: 43.59015321972929
r2 score: 0.7145750298588391
MRE: 14.416121173057729
MAPE: 14.416122436523438


predict_3
-------------

In [99]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 32.08628869448182
mean_squared_error: 1893.274627624495
rmse: 43.51177573513284
r2 score: 0.71567418325279
MRE: 14.483410788443141
MAPE: 14.483410120010376


predict_4
-------------

In [100]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 32.27493099966342
mean_squared_error: 1891.5327734096263
rmse: 43.49175523486752
r2 score: 0.7159999836782436
MRE: 14.60811440221369
MAPE: 14.608114957809448


vanilla_demand_predict_24hr
==========


In [17]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_96step` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [106]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])

In [107]:
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 29.309350824083417
mean_squared_error: 1859.7163218298017
rmse: 43.124428365252584
r2 score: 0.7206415091821546
MRE: 12.951875748292696
MAPE: 12.95187622308731


In [23]:
c = positive_negative_error(demand_dataframe_1['demand_quarter'].astype('float32'),demand_dataframe_1['prediction_1'].astype('float32'), demand_dataframe_1)

pd.DataFrame(data=c).to_csv('第一次預測-20220717.csv', index=False)

predict_2
-------------

In [104]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 30.30962314939434
mean_squared_error: 1916.9962711978465
rmse: 43.78351597573962
r2 score: 0.7121117440877747
MRE: 13.512744803544196
MAPE: 13.512744009494781


In [309]:
c = positive_negative_error(demand_dataframe_2['demand_quarter'].astype('float32'),demand_dataframe_2['prediction_2'].astype('float32'), demand_dataframe_2)

pd.DataFrame(data=c).to_csv('畢業典禮24小時誤差-2.csv', index=False)

predict_3
-------------

In [25]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 28.860824638168964
mean_squared_error: 1802.31597694379
rmse: 42.45369214737147
r2 score: 0.7293952428081183
MRE: 12.867064249405487
MAPE: 12.867063283920288


In [311]:
c = positive_negative_error(demand_dataframe_3['demand_quarter'].astype('float32'),demand_dataframe_3['prediction_3'].astype('float32'), demand_dataframe_3)

In [312]:
pd.DataFrame(data=c).to_csv('畢業典禮24小時誤差-3.csv', index=False)

predict_4
-------------

In [28]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 28.820239057239057
mean_squared_error: 1787.4249957912455
rmse: 42.27794928554654
r2 score: 0.7316835897243199
MRE: 12.920228000183828
MAPE: 12.920227646827698


vanilla_demand_predict_self_attention_y1
==========

In [10]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_y1` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [11]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])

mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()

print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 13.851632927702218
mean_squared_error: 354.20372095060844
rmse: 18.82030076674144
r2 score: 0.9420205719919207
MRE: 10.387336827486083
MAPE: 10.38733646273613


predict_2
-------------

In [12]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 12.566932195624785
mean_squared_error: 289.64770471881803
rmse: 17.019039476974545
r2 score: 0.9525859827002816
MRE: 9.546188458990738
MAPE: 9.54618826508522


predict_3
-------------

In [13]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 11.031901325773845
mean_squared_error: 220.50862219454226
rmse: 14.849532726471303
r2 score: 0.9639020378816017
MRE: 8.48476728265607
MAPE: 8.484767377376556


predict_4
-------------

In [14]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 9.15638193688792
mean_squared_error: 148.8091353559361
rmse: 12.198734989987122
r2 score: 0.9756384500730965
MRE: 7.228865645856382
MAPE: 7.228865474462509


vanilla_demand_predict_self_attention_y2
==========

In [16]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_y2` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [17]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 13.584818897637794
mean_squared_error: 373.08266157766644
rmse: 19.315347824402917
r2 score: 0.9389302877452796
MRE: 9.94492026561987
MAPE: 9.944919496774673


predict_2
-------------

In [18]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 12.187388901614938
mean_squared_error: 295.1820348671401
rmse: 17.18086245993315
r2 score: 0.9516800379228161
MRE: 8.99160868303638
MAPE: 8.991608768701553


predict_3
-------------

In [19]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 10.597426624287719
mean_squared_error: 219.05633732210865
rmse: 14.80055192626642
r2 score: 0.9641397815298477
MRE: 7.882929362477507
MAPE: 7.882929593324661


predict_4
-------------

In [20]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 8.72536739018384
mean_squared_error: 145.44294987686845
rmse: 12.059973046274543
r2 score: 0.9761895284421453
MRE: 6.593034724364098
MAPE: 6.593034416437149


vanilla_demand_predict_self_attention_y3
==========

In [21]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_y3` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [22]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 14.723139584824624
mean_squared_error: 446.69548145168216
rmse: 21.13517166837502
r2 score: 0.926880642476442
MRE: 10.415056855948713
MAPE: 10.415057092905045


predict_2
-------------

In [23]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 13.029853109609439
mean_squared_error: 338.86171513285996
rmse: 18.4081969549671
r2 score: 0.9445298721109534
MRE: 9.399844505860843
MAPE: 9.399844706058502


predict_3
-------------

In [24]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 11.208039400967845
mean_squared_error: 242.27685182258114
rmse: 15.565244997191055
r2 score: 0.9603385095230403
MRE: 8.249683269321768
MAPE: 8.249683678150177


predict_4
-------------

In [25]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 9.007921653971708
mean_squared_error: 150.33187280797205
rmse: 12.26098987879739
r2 score: 0.9753891626595609
MRE: 6.8781157043509085
MAPE: 6.8781159818172455


vanilla_demand_predict_self_attention_4t
=========

In [10]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_4t` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [11]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 23.80705798138869
mean_squared_error: 1132.4505445382963
rmse: 33.65190253965289
r2 score: 0.8146297428961113
MRE: 20.47220423046903
MAPE: 20.4722061753273


predict_2
-------------

In [12]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 18.869475718703473
mean_squared_error: 682.478191681938
rmse: 26.12428356303648
r2 score: 0.8882814113148206
MRE: 16.178117181787897
MAPE: 16.17811620235443


predict_3
------------

In [13]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 15.037726999398679
mean_squared_error: 431.29784208115
rmse: 20.767711527300015
r2 score: 0.9293951728043692
MRE: 13.047348414722162
MAPE: 13.047347962856293


predict_4
-------------

In [14]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 11.142911058931332
mean_squared_error: 237.01389500601343
rmse: 15.395255600541793
r2 score: 0.9611984450904307
MRE: 9.619345422931385
MAPE: 9.619346261024475


vanilla_demand_predict_self_attention_48t
=========

In [16]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_48t` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

predict_1
-------------

In [17]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 15.290934001431639
mean_squared_error: 466.80508045526125
rmse: 21.605672413865328
r2 score: 0.9235889123823305
MRE: 11.110780945293262
MAPE: 11.110781133174896


predict_2
-------------

In [18]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 13.830771961974571
mean_squared_error: 387.19907404077424
rmse: 19.67737467348666
r2 score: 0.9366172654023747
MRE: 10.202945504931234
MAPE: 10.20294576883316


predict_3
-------------

In [19]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 11.894026572745755
mean_squared_error: 286.7628974229018
rmse: 16.934075038894264
r2 score: 0.9530560025040588
MRE: 8.864461860158784
MAPE: 8.86446163058281


predict_4
-------------

In [20]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 9.202364698470877
mean_squared_error: 167.4201184410973
rmse: 12.9390926436554
r2 score: 0.97259164523458
MRE: 6.909778964090846
MAPE: 6.909778714179993


vanilla_demand_predict_self_attention_192t
=========

In [31]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at`, `dwwd`.`data_hour`, `dwwd`.`data_weekday` 
                    FROM  `vanilla_demand_predict_self_attention_192t` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`data_date` >='{}' AND `dwwd`.`data_date` <='{}' 
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

(2003, "Can't connect to MySQL server on 'localhost' ([WinError 10061] 無法連線，因為目標電腦拒絕連線。)")


predict_1
-------------

In [32]:
demand_dataframe_1 = demand_dataframe.dropna(subset=['prediction_1'])
mean_rel_error = demand_dataframe_1.apply(lambda demand_dataframe_1: rel_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']), axis=1).sum()/demand_dataframe_1['prediction_1'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1'])))
print("r2 score:",r2_score(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_1['demand_quarter'],demand_dataframe_1['prediction_1']))

mean_absolute_error: 15.290934001431639
mean_squared_error: 466.80508045526125
rmse: 21.605672413865328
r2 score: 0.9235889123823305
MRE: 11.110780945293262
MAPE: 11.110781133174896


predict_2
-------------

In [33]:
demand_dataframe_2 = demand_dataframe.dropna(subset=['prediction_2'])
mean_rel_error = demand_dataframe_2.apply(lambda demand_dataframe_2: rel_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']), axis=1).sum()/demand_dataframe_2['prediction_2'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2'])))
print("r2 score:",r2_score(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_2['demand_quarter'],demand_dataframe_2['prediction_2']))

mean_absolute_error: 13.830771961974571
mean_squared_error: 387.19907404077424
rmse: 19.67737467348666
r2 score: 0.9366172654023747
MRE: 10.202945504931234
MAPE: 10.20294576883316


predict_3
-------------

In [34]:
demand_dataframe_3 = demand_dataframe.dropna(subset=['prediction_3'])
mean_rel_error = demand_dataframe_3.apply(lambda demand_dataframe_3: rel_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']), axis=1).sum()/demand_dataframe_3['prediction_3'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3'])))
print("r2 score:",r2_score(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_3['demand_quarter'],demand_dataframe_3['prediction_3']))

mean_absolute_error: 11.894026572745755
mean_squared_error: 286.7628974229018
rmse: 16.934075038894264
r2 score: 0.9530560025040588
MRE: 8.864461860158784
MAPE: 8.86446163058281


predict_4
-------------

In [35]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 9.202364698470877
mean_squared_error: 167.4201184410973
rmse: 12.9390926436554
r2 score: 0.97259164523458
MRE: 6.909778964090846
MAPE: 6.909778714179993


平日假日各24小時誤差
=========

In [18]:
demand_dataframe_5 = demand_dataframe.dropna(subset=['prediction_4'])
demand_dataframe_5_weekday =  demand_dataframe_5[demand_dataframe_5['data_weekday'].isin([1, 2, 3, 4, 5])]
demand_dataframe_5_holiday =  demand_dataframe_5[demand_dataframe_5['data_weekday'].isin([6, 7])]

In [19]:
mean_rel_error_weekday = demand_dataframe_5_weekday.apply(lambda demand_dataframe_5_weekday: rel_error(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']), axis=1).sum()/demand_dataframe_5_weekday['prediction_4'].count()

In [20]:
mean_rel_error_holiday = demand_dataframe_5_holiday.apply(lambda demand_dataframe_5_holiday: rel_error(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']), axis=1).sum()/demand_dataframe_5_holiday['prediction_4'].count()

In [21]:
mean_rel_error_weekday, mean_rel_error_holiday

(0.10826997558141713, 0.17302929238209513)

In [73]:
weekday_error = positive_negative_error(demand_dataframe_5_weekday['demand_quarter'].astype('float32'),demand_dataframe_5_weekday['prediction_4'].astype('float32'), demand_dataframe_5_weekday)
holiday_error = positive_negative_error(demand_dataframe_5_holiday['demand_quarter'].astype('float32'),demand_dataframe_5_holiday['prediction_4'].astype('float32'), demand_dataframe_5_holiday)


In [74]:
pd.DataFrame(data=weekday_error).to_csv('C:/Users/KDD_179/erica/training-result/平日每小時誤差-4.csv', index=False)
pd.DataFrame(data=holiday_error).to_csv('C:/Users/KDD_179/erica/training-result/假日每小時誤差-4.csv', index=False)

平日
-------------

In [80]:
mean_rel_error = demand_dataframe_5_weekday.apply(lambda demand_dataframe_5_weekday: rel_error(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']), axis=1).sum()/demand_dataframe_5_weekday['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_5_weekday['demand_quarter'],demand_dataframe_5_weekday['prediction_4']))

mean_absolute_error: 25.989800995024876
mean_squared_error: 1331.482372437811
rmse: 36.48948303878545
r2 score: 0.8281586117325962
MRE: 10.826997558141713
MAPE: 10.826997458934784


假日
-------------

In [81]:
mean_rel_error = demand_dataframe_5_holiday.apply(lambda demand_dataframe_5_holiday: rel_error(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']), axis=1).sum()/demand_dataframe_5_holiday['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_5_holiday['demand_quarter'],demand_dataframe_5_holiday['prediction_4']))

mean_absolute_error: 34.74646875
mean_squared_error: 2742.0548634375
rmse: 52.36463370861578
r2 score: -0.06890699392496513
MRE: 17.302929238209515
MAPE: 17.302928864955902


----------------

In [31]:
demand_dataframe_4

,prediction_1,prediction_2,prediction_3,prediction_4,demand_quarter,created_at,data_hour
0,144.80,145.50,145.25,145.25,156.8,2021-05-01 00:00:00,0
1,143.37,143.12,143.12,146.12,145.2,2021-05-01 00:15:00,0
2,141.06,141.07,143.39,143.71,157.2,2021-05-01 00:30:00,0
3,139.09,140.78,141.11,140.77,137.2,2021-05-01 00:45:00,0
4,137.49,137.79,137.43,138.05,130.4,2021-05-01 01:00:00,1
...,...,...,...,...,...,...,...
2965,212.15,211.15,207.32,208.21,203.6,2021-05-31 23:00:00,23
2966,205.84,201.78,202.80,209.49,195.2,2021-05-31 23:15:00,23
2967,197.37,198.50,205.08,203.76,195.2,2021-05-31 23:30:00,23
2968,195.36,201.55,200.35,198.75,196.0,2021-05-31 23:45:00,23


In [ ]:
c = positive_negative_error(demand_dataframe_4['demand_quarter'].astype('float32'),demand_dataframe_4['prediction_4'].astype('float32'), demand_dataframe_4)

In [32]:
demand_avg = demand_avg(demand_dataframe_4['demand_quarter'].astype('float32'), demand_dataframe_4)

TypeError: demand_avg() missing 1 required positional argument: 'dataframe'

In [178]:
pd.DataFrame(data=demand_avg).to_csv('demand-avg.csv', index=False)

In [196]:
a, b = positive_negative_error(demand_dataframe_4['demand_quarter'].astype('float32'),demand_dataframe_4['prediction_4'].astype('float32'), demand_dataframe_4)

In [206]:
pd.DataFrame(data=a).to_csv('positive-error-relative-rate-2.csv', index=False)

In [207]:
pd.DataFrame(data=b).to_csv('negative-error-relative-rate-2.csv', index=False)

In [233]:
a = positive_negative_error_weekday(demand_dataframe_4['demand_quarter'].astype('float32'),demand_dataframe_4['prediction_4'].astype('float32'), demand_dataframe_4)

In [236]:
a, b = positive_negative_error(demand_dataframe_4['demand_quarter'].astype('float32'),demand_dataframe_4['prediction_4'].astype('float32'), demand_dataframe_4)

In [237]:
pd.DataFrame(data=a).to_csv('WEEKDAY-positive-error-relative-rate-2.csv', index=False)

In [238]:
pd.DataFrame(data=b).to_csv('WEEKDAY-negative-error-relative-rate-2.csv', index=False)

In [314]:
c = positive_negative_error(demand_dataframe_4['demand_quarter'].astype('float32'),demand_dataframe_4['prediction_4'].astype('float32'), demand_dataframe_4)

In [315]:
pd.DataFrame(data=c).to_csv('畢業典禮24小時誤差-4.csv', index=False)

linear_regression_predict
==========

In [38]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at` 
                    FROM  `linear_regression_predict` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`datetime` >= '{}' AND `dwwd`.`datetime` <= '{}'
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [39]:
demand_dataframe_4 = demand_dataframe.dropna(subset=['prediction_4'])
mean_rel_error = demand_dataframe_4.apply(lambda demand_dataframe_4: rel_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']), axis=1).sum()/demand_dataframe_4['prediction_4'].count()
print("mean_absolute_error:",mean_absolute_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe_4['demand_quarter'],demand_dataframe_4['prediction_4']))

mean_absolute_error: 45.42954219337206
mean_squared_error: 3391.2735227195085
rmse: 58.23464194720792
r2 score: 0.6018725393989584
MRE: 22.22743427258885
MAPE: 22.227434813976288


---------------

bidirectional_demand_predict
==========


In [10]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at` 
                    FROM  `bidirectional_demand_predict` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`datetime` >= '{}' AND `dwwd`.`datetime` <= '{}'
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [11]:
demand_dataframe

,prediction_1,prediction_2,prediction_3,prediction_4,demand_quarter,created_at
0,None,None,None,162.10,187.2,2020-06-01 00:15:00
1,None,None,157.88,149.95,168.4,2020-06-01 00:30:00
2,None,153.64,146.13,150.90,148.8,2020-06-01 00:45:00
3,153.64,146.78,151.69,160.55,152.0,2020-06-01 01:00:00
4,143.11,147.90,156.57,166.81,150.0,2020-06-01 01:15:00
...,...,...,...,...,...,...
17556,160.54,159.30,157.61,155.54,125.6,2020-11-30 23:00:00
17557,157.46,155.77,153.71,149.26,130.0,2020-11-30 23:15:00
17558,154.04,151.98,147.60,145.08,132.4,2020-11-30 23:30:00
17559,150.36,146.05,143.53,140.92,126.8,2020-11-30 23:45:00


In [4]:
mean_rel_error = demand_dataframe.apply(lambda demand_dataframe: rel_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']), axis=1).sum()/demand_dataframe['prediction_4'].count()

NameError: name 'demand_dataframe' is not defined

In [81]:
print("mean_absolute_error:",mean_absolute_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))

mean_absolute_error: 40.754336939102565
mean_squared_error: 3460.847926181891
rmse: 58.82897182665945
r2 score: 0.10850199480902212
MRE: 17.502481617928346
MAPE: 22.147032618522644


stacked_demand_predict
==========


In [155]:
# 撈取需量資料 [B]
try: 
    conn  =  pymysql.connect ( host = ' ' ,  user = ' ' ,  passwd = " " ,  db = ' ' ) 
    cur  =  conn.cursor() 
    select_sql = '''SELECT `dp`.`prediction_1`, `dp`.`prediction_2`, `dp`.`prediction_3`, `dp`.`prediction_4`, `dwwd`.`demand_quarter`, `dp`.`created_at` 
                    FROM  `stacked_demand_predict` AS `dp`
                    INNER JOIN `demand_with_weather_data` AS `dwwd` ON(`dp`.`created_at` = `dwwd`.`datetime`)
                    WHERE `dwwd`.`datetime` >= '{}' AND `dwwd`.`datetime` <= '{}'
                    ORDER BY `dp`.`created_at` '''.format(predict_begin_date, predict_end_date)
    result_object = cur.execute(select_sql)
    results_values_list = cur.fetchall()
    result_key_list = [i[0] for i in cur.description]
    
    demand_dataframe = pd.DataFrame(results_values_list)
    demand_dataframe.columns = result_key_list
    cur.close () 
    conn.close()
except Exception as e:
    print(e)
# 撈取需量資料 [E]

In [156]:
mean_rel_error = demand_dataframe.apply(lambda demand_dataframe: rel_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']), axis=1).sum()/demand_dataframe['prediction_4'].count()

In [157]:
print("mean_absolute_error:",mean_absolute_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("mean_squared_error:",mean_squared_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("rmse:",sqrt(mean_squared_error(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4'])))
print("r2 score:",r2_score(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))
print("MRE:",mean_rel_error*100)
print("MAPE:",mape(demand_dataframe['demand_quarter'],demand_dataframe['prediction_4']))

mean_absolute_error: 18.05804159445407
mean_squared_error: 619.1829658578856
rmse: 24.88338734694064
r2 score: 0.6113324302307327
MRE: 13.561764560353609
MAPE: 13.561764359474182
